In [1]:
import pandas as pd
import os
import csv
os.chdir('Resources/')

In [2]:
df = pd.read_csv('0_Raw_Data.csv')
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,5/2/2010,24924.50,False
1,1,1,12/2/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,5/3/2010,21827.90,False


In [3]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
unique_dates = df['Date'].sort_values().unique()
print(len(unique_dates))
df

143


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False


In [4]:
df['IsHoliday'] = df['IsHoliday'].astype(int)
df['Store'] = pd.to_numeric(df['Store'])
df['Dept'] = pd.to_numeric(df['Dept'])
df['Weekly_Sales'] = pd.to_numeric(df['Weekly_Sales'])
df

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,0
1,1,1,2010-02-12,46039.49,1
2,1,1,2010-02-19,41595.55,0
3,1,1,2010-02-26,19403.54,0
4,1,1,2010-03-05,21827.90,0
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,0
421566,45,98,2012-10-05,628.10,0
421567,45,98,2012-10-12,1061.02,0
421568,45,98,2012-10-19,760.01,0


In [5]:
df = df.sort_values(by=['Date', 'Store', 'Dept']).reset_index(drop=True)
df

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,0
1,1,2,2010-02-05,50605.27,0
2,1,3,2010-02-05,13740.12,0
3,1,4,2010-02-05,39954.04,0
4,1,5,2010-02-05,32229.38,0
...,...,...,...,...,...
421565,45,93,2012-10-26,2487.80,0
421566,45,94,2012-10-26,5203.31,0
421567,45,95,2012-10-26,56017.47,0
421568,45,97,2012-10-26,6817.48,0


In [6]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df.drop(columns=['Date'], inplace=True)

df['Year'] = pd.to_numeric(df['Year'])
df['Month'] = pd.to_numeric(df['Month'])
df['Day'] = pd.to_numeric(df['Day'])
df

,Store,Dept,Weekly_Sales,IsHoliday,Year,Month,Day
0,1,1,24924.50,0,2010,2,5
1,1,2,50605.27,0,2010,2,5
2,1,3,13740.12,0,2010,2,5
3,1,4,39954.04,0,2010,2,5
4,1,5,32229.38,0,2010,2,5
...,...,...,...,...,...,...,...
421565,45,93,2487.80,0,2012,10,26
421566,45,94,5203.31,0,2012,10,26
421567,45,95,56017.47,0,2012,10,26
421568,45,97,6817.48,0,2012,10,26


In [7]:
df.to_csv('1_Structured_Data.csv', index=False)

In [8]:
stores = sorted(df['Store'].unique()) 

store_dfs = {store: df[df['Store'] == store].copy() for store in stores}

store_keys = {
    store: set(zip(store_dfs[store]['Dept'], store_dfs[store]['Year'], store_dfs[store]['Month'], store_dfs[store]['Day']))
    for store in stores
}

common_keys = set.intersection(*store_keys.values())

print(f"Number of common records across all stores: {len(common_keys)}")

for store in stores:
    df_store = store_dfs[store]
    mask = df_store.apply(lambda row: (row['Dept'], row['Year'], row['Month'], row['Day']) in common_keys, axis=1)
    filtered_df = df_store[mask].copy()
    
    filtered_df.reset_index(drop=True, inplace=True)
    
    filename = f'Store_{store}_Filtered.csv'
    filtered_df.to_csv(filename, index=False)

Number of common records across all stores: 4417


In [9]:
import pandas as pd

stores = range(1, 46)

for store in stores:
    df = pd.read_csv(f'Store_{store}_Filtered.csv')
    df.sort_values(by=['Year', 'Month', 'Day'], inplace=True)
    df['Week'] = df.groupby(['Year', 'Month'])['Day'].rank(method='dense').astype(int)
    df.drop(columns='Day', inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.to_csv(f'Store_{store}_Filtered.csv', index=False)

In [10]:
for store in stores:
    df = pd.read_csv(f'Store_{store}_Filtered.csv')
    year_months = sorted(df[['Year', 'Month']].drop_duplicates().apply(tuple, axis=1).tolist())

    month_dfs = {
        (year, month): df[(df['Year'] == year) & (df['Month'] == month)].copy()
        for (year, month) in year_months
    }

    month_keys = {
        key: set(zip(month_df['Dept'], month_df['Week']))
        for key, month_df in month_dfs.items()
    }

    common_keys = set.intersection(*month_keys.values())

    for key, month_df in month_dfs.items():
        mask = month_df.apply(lambda row: (row['Dept'], row['Week']) in common_keys, axis=1)
        filtered_df = month_df[mask].copy()
        filtered_df.reset_index(drop=True, inplace=True)

        year, month = key
        filename = f'Store_{store}_{year}_{month}_Filtered.csv'
        filtered_df.to_csv(filename, index=False)

In [11]:
import os
import glob

files_to_delete = []

for i in range(1, 46):
    files_to_delete.append(f'Store_{i}_Filtered.csv')

for file in files_to_delete:
    if os.path.isfile(file):
        os.remove(file)

In [12]:
import pandas as pd
import glob
import os
from collections import defaultdict

for j in range(0, 9):
    files = []
    for i in range(1 + 5*j, 6 + 5*j):
        files.extend(glob.glob(f"Store_{i}_*_filtered.csv"))

    groups = defaultdict(list)

    for file in files:
        basename = os.path.basename(file)
        parts = basename.split('_')
        
        year_month = f"{parts[2]}_{parts[3]}"
        groups[year_month].append(file)

    for year_month, file_list in groups.items():
        dfs = [pd.read_csv(f) for f in file_list]
        merged_df = pd.concat(dfs, ignore_index=True)
        merged_filename = f"1_{j+1}_Client_Data_{year_month}.csv"
        merged_df.to_csv(merged_filename, index=False)

In [13]:
import os
import glob

files_to_delete = glob.glob("Store*")

for file in files_to_delete:
    if os.path.isfile(file):
        os.remove(file)